In [1]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset

In [2]:
import torch

In [3]:
import sys
sys.path.append('../scripts')

from dataloader import (get_train_mapping, BaseDataset , get_dataloaders)

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
from IPython.display import Audio
import librosa
import librosa.display

def show_audio(y,sr=16000):
    librosa.display.waveplot(y, sr=sr)

    spec= load_mel.get_spectrogram(y,sr,apply_denoise=False,return_audio=False)
    load_mel.plot_feature(spec)
    
    return Audio(y,rate=sr)

In [5]:
root_dir= '/home/jupyter/rfcx/data/train'
df= pd.read_csv('/home/jupyter/rfcx/data/train_tp.csv')


mapping= get_train_mapping(df)
train_dataset = BaseDataset(root_dir, mapping,  enable_mixup=True, enable_aug=False)
test_dataset = BaseDataset(root_dir, mapping,  enable_mixup=False, enable_aug=False)

In [19]:
batch_size=3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
dls= get_dataloaders(train_dataset, test_dataset, batch_size,device)

training: number of docs : 378
evaluation: number of docs : 378


In [21]:
b = dls.one_batch()

In [28]:
train, test = b

In [29]:
train[0].shape

torch.Size([3, 4801, 80])

In [30]:
test.shape

torch.Size([3, 24])